# Fine tune Vit built by Google Brain, which trained on ImageNet-21k, for Fashion-MNIST

In [1]:
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import requests

model_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_path)
#model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

/home/kang5647/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-12 15:58:36.975887: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 15:58:37.088967: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-12 15:58:37.088983: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on yo

In [2]:
processor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [3]:
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
import torch 
import torchvision.transforms as transforms
from torch.utils.data import Dataset

class CustomFashionMNIST(Dataset):
    def __init__(self, dataset):
        """
        Wrap a given dataset to adjust the format of its items.
        :param dataset: The original dataset to wrap.
        """
        self.dataset = dataset

    def __getitem__(self, idx):
        """
        Returns a single item from the dataset, adjusting its format.
        """
        # Fetch the original item
        item, label = self.dataset[idx]
        
        # Assuming 'item' is a dictionary containing 'pixel_values',
        # adjust the format to include 'labels' in the dictionary.
        item['labels'] = label

        return item

    def __len__(self):
        """
        Return the total number of items in the dataset.
        """
        return len(self.dataset)

# Apply your transforms as before
transform = transforms.Compose([
    transforms.Lambda(lambda pil_img: pil_img.convert("RGB")),
    transforms.Lambda(lambda pil_img: processor(pil_img, return_tensors='pt'))
])

# Initialize the original datasets
original_train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
original_test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Wrap the original datasets
train_dataset = CustomFashionMNIST(original_train_dataset)
test_dataset = CustomFashionMNIST(original_test_dataset)

In [4]:
pixel_value = torch.squeeze(train_dataset[0]['pixel_values'])
pixel_value.shape

torch.Size([3, 224, 224])

In [5]:
def collate_fn(batch):
    pixel_values = torch.stack([torch.squeeze(x['pixel_values'], dim=0) for x in batch])
    labels = torch.tensor([x['labels'] for x in batch])
    
    return {
        'pixel_values': pixel_values,
        'labels': labels
    }

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


/home/kang5647/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [7]:
from transformers import ViTForImageClassification

labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

model = ViTForImageClassification.from_pretrained(
    model_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

model = ViTForImageClassification.from_pretrained("./vit-fashion-mnist")

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-fashion-mnist",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [13]:
from accelerate import Accelerator
accelerator = Accelerator()

device = accelerator.device
model,train_dataset, test_dataset= accelerator.prepare(
    model, train_dataset, test_dataset
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor,
)

/home/FYP/poon0064/.conda/envs/RunJupyter/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss,Accuracy
100,0.638900,0.596072,0.817400
200,0.445700,0.463095,0.850100
300,0.500100,0.495619,0.831700
400,0.322700,0.412769,0.860900
500,0.392200,0.347485,0.880300
600,0.336400,0.360804,0.879300
700,0.251500,0.334698,0.887600
800,0.453200,0.382384,0.868300
900,0.331800,0.295076,0.898100
1000,0.411600,0.416310,0.856400


## Resume from training error

In [16]:
train_results = trainer.train(resume_from_checkpoint = True)
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Could not locate the best model at ./vit-fashion-mnist/checkpoint-11300/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


Step,Training Loss,Validation Loss


***** train metrics *****
  epoch                    =           4.0
  total_flos               = 17322051354GF
  train_loss               =           0.0
  train_runtime            =    0:00:00.05
  train_samples_per_second =    4647793.04
  train_steps_per_second   =    290487.065


## Predict samples of test dataset

In [8]:
metrics = trainer.evaluate(test_dataset)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

NameError: name 'trainer' is not defined

In [34]:
import random 

random.seed(42)

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

test_samples = []
for sample in random.sample(list(test_dataset), k=9):
    test_samples.append(sample)


[{'pixel_values': tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]],
 
          [[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]],
 
          [[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]]]]), 'labels': 5},
 {'pixel_values': tensor([[[[

In [39]:
preds, labels, metrics = trainer.predict(test_samples, metric_key_prefix="predict")
trainer.log_metrics("predict", metrics)
trainer.save_metrics("predict", metrics)

pred_classes = predictions = np.argmax(preds, axis=1)

pred_classes, labels

***** predict metrics *****
  predict_accuracy           =        1.0
  predict_loss               =     0.0112
  predict_runtime            = 0:00:00.04
  predict_samples_per_second =    184.041
  predict_steps_per_second   =     40.898


(array([5, 1, 7, 4, 3, 0, 4, 7, 1]), array([5, 1, 7, 4, 3, 0, 4, 7, 1]))